## Cal-Adapt Analytics Engine Data processing script for DWR

This notebook downloads climate data for pr, tasmax, and tasmin, processes the data to monthly time step, weights each sub-basin of interest by Area and Flow and determines the average monthly value for the entire watershed. The average weighted results for Area and Flow are then processed into a monthly rolling average series for pr, tasmax, tasmin, and tasave.

The output format of processed climate data are listed below:
* Each subasin is stored stored to exported to CSV without the weights with file name format {source_id}_{experiment_id}_{member_id}_{basin_id}_19.csv.  
* The Area and Flow weighted value are exported to CSV files in seperate directories with the file name format {source_id}_{experiment_id}_{member_id}_19{weight_type}Weighted.csv.
* The monthly rolling average results are exported to CSV files in seperate diretories based on inputs weighting.  The file format is {source_id}_{member_id}_{experiment_id}_30yrAve.csv.


Note: All SSPs/realization(member_1d) must have the corrisponding historical/realization(member_1d) otherwise the 30YrAve post-proccesing will throw an error. 

Inputs listed below:
* data/Basin_Weights.csv: Constains Basin the data for the basin weighting.
* data/GCM_Run_List_#-#.csv: Constains a run list for all GCMs of interest.  Note: Each SSP must contain corrisponding historical for 30 year rolling average calc.

Known Issues:  
* When running a large run list, if you screen is locked or browser focus is lost the kernal will lose connection with Dask array.  To avoid this, you can either convert to a script and run in a terminal window like `tmux` or `screen`, or simply shorten the run list.


In [1]:
import csv
from io import StringIO
import os
import zipfile

import pandas as pd
import intake
import numpy as np
import xarray as xr
import rioxarray
import dask
import panel as pn
from dask.distributed import get_client

pn.extension()
xr.set_options(keep_attrs=True)
dask.config.set({"array.slicing.split_large_chunks": True})


In [2]:
# VARIABLES
#Use these cordinates to clip around the watershed of interest.
bbox = {
    "maxy": 42.432494,
    "miny": 34.775317,
    "minx": -123.097421,
    "maxx": -117.980799,
}

#run_list_path = "data/GCM_Run_List_1-43.csv"
#file_zip = "GCM_1-43.zip"

run_list_path = "data/RunLists/GCM_Run_List_1-19.csv"
file_zip = "GCM_1-19.zip"

run_list_path = "data/RunLists/GCM_Run_List_20-43.csv"
file_zip = "GCM_20-43.zip"

run_list_path = "data/RunLists/GCM_Run_List_44-82.csv"
file_zip = "GCM_44-82.zip"

run_list_path = "data/RunLists/GCM_Run_List_83-100.csv"
file_zip = "GCM_83-100.zip"

run_list_path = "data/RunLists/GCM_Run_List_Add_1-22.csv"
file_zip = "GCM_Add_1-22.zip"

run_list_path = "data/RunLists/GCM_Run_List_Add_23-47.csv"
file_zip = "GCM_Add_23-47.zip"

run_list_path = "data/RunLists/GCM_Run_List_Add_48-75.csv"
file_zip = "GCM_Add_48-75.zip"

#run_list_path = "data/GCM_Run_List_Test_Rolling.csv"
#file_zip = "GCM_Rolling_Test.zip"

basin_weights_csv = "data/Basin_Weights.csv"
esm_datastore = "https://cadcat.s3.amazonaws.com/cae-collection.json"
output_folder = "outputs"
mask_path = "mask/mask.npy"
dir_area_weighted = 'AREA_WEIGHTED_CENTRALVALLEY'
dir_flow_weighted = 'FLOW_WEIGHTED_CENTRALVALLEY'
dir_non_weighted = 'NON_WEIGHTED_CENTRALVALLEY'
dir_individual = 'INDIVIDUAL_BASIN_LOCA2'
dir_area_weighted_rolling = 'AREA_WEIGHTED_30_YEAR_ROLLING_AVE_CENTRALVALLEY'
dir_flow_weighted_rolling = 'FLOW_WEIGHTED_30_YEAR_ROLLING_AVE_CENTRALVALLEY'


This loads the dask client for faster processing.  If client is running, do not run this cell.

In [3]:
#Load dask Area for faster computing. 
#Note, this will take some time but in long run processing should be faster when compute is called.
# Check if there is already a dask client running
try:
    client = get_client()
    print("Using existing dask client")
except ValueError:
    # No client found, create a new one
    cluster = dask.distributed.LocalCluster(
        n_workers=16,  # Adjust the number of workers as needed
        threads_per_worker=1,  # Use one thread per worker
        memory_limit='2GB'  # Adjust memory limit per worker as needed
    )
    cluster.adapt(minimum=0, maximum=16)
    client = cluster.get_client()
    print("Created new dask client")

# Get client link
client

Created new dask client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45569,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33895,Total threads: 1
Dashboard: http://127.0.0.1:43257/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:43193,


2025-08-05 12:51:52,439 - tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x7fb89a41b740>, <Task finished name='Task-75112' coro=<BaseTCPListener._handle_stream() done, defined at /srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/comm/tcp.py:655> exception=ValueError('invalid operation on non-started TCPListener')>)
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/tornado/tcpserver.py", line 387, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
                                           ^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/comm/tcp.py", line 661, in _handle_stream
    logger.debug("Incoming connection from %r to %r", address, self

In [4]:
def get_region_dict() -> dict:
    """
    Gets flow and area weights for each region from the basin weights CSV file.

    Returns
    -------
    dict
        Dictionary with region ID as key and dictionary containing region_name,
        flow_ratio, and area_ratio as values.
    """
    region_dict = {}
    with open(basin_weights_csv, "r") as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            region_dict[int(row["ID"])] = {
                                "region_name": row["Regions"],
                                "flow_ratio": float(row["Flow Ratio"]),
                                "area_ratio": float(row["Area Ratio"]),
                            }
    return region_dict

In [5]:
def get_model_params(run_list_path: str) -> list[dict]:
    """
    Read each set of model parameters into dictionary from csv file.
    
    Parameters
    ----------
    run_list_path : str
        Path to CSV file containing model parameters with columns for
        activity_id, institution_id, source_id, experiment_id, table_id,
        grid_label, member_id, start_year, and end_year.
    
    Returns
    -------
    list[dict]
        List of dictionaries where each dictionary contains model parameters
        for a single model run with keys corresponding to CSV column headers.
    
    Examples
    --------
    >>> params = get_model_params("data/GCM_Run_List_1-19.csv")
    >>> len(params)
    19
    >>> params[0].keys()
    dict_keys(['activity_id', 'institution_id', 'source_id', ...])
    """
    model_params = []
    with open(run_list_path, "r") as src:
        d = csv.DictReader(src)
        for row in d:
            model_params.append(row)
    return model_params

In [6]:
def slice_by_time_years_dataset(ds: xr.Dataset, startyear: str, endyear: str) -> xr.Dataset:
    """
    Slice the dataset to years of interest.
    
    Parameters
    ----------
    ds : xarray.Dataset
        Input dataset containing time dimension to be sliced
    startyear : str
        Start year for slicing in string format (e.g., '1950')
    endyear : str
        End year for slicing in string format (e.g., '2014')
    
    Returns
    -------
    xarray.Dataset
        Dataset sliced to the specified time range
    
    Examples
    --------
    >>> ds = xr.Dataset({'temp': (['time'], np.random.rand(100))})
    >>> sliced_ds = slice_by_time_years_dataset(ds, '1950', '2014')
    >>> # Returns dataset with time dimension sliced from 1950 to 2014
    """
    ds = ds.sel(
        time=slice(str(startyear), str(endyear))
        )
    return ds

In [7]:
def convert_daily_to_monthly_dataset(ds: xr.Dataset) -> xr.Dataset:
    """
    Convert daily climate data to monthly aggregated values.
    
    Converts daily precipitation from kg/m2/s to monthly accumulated mm,
    and daily temperature from Kelvin to monthly average Celsius.
    
    Parameters
    ----------
    ds : xarray.Dataset
        Input dataset containing daily climate data with variables:
        - pr: precipitation in kg/m2/s
        - tasmin: minimum temperature in Kelvin
        - tasmax: maximum temperature in Kelvin
    
    Returns
    -------
    xarray.Dataset
        Dataset with monthly aggregated climate data:
        - pr: monthly accumulated precipitation in mm/mon
        - tasmin: monthly average minimum temperature in degC
        - tasmax: monthly average maximum temperature in degC
    
    Notes
    -----
    Precipitation conversion: kg/m2/s * 86400 s/day = mm/day
    Temperature conversion: K - 273.15 = degC
    
    Examples
    --------
    >>> daily_ds = xr.Dataset({
    ...     'pr': (['time', 'lat', 'lon'], np.random.rand(365, 10, 10)),
    ...     'tasmin': (['time', 'lat', 'lon'], np.random.rand(365, 10, 10) + 273.15),
    ...     'tasmax': (['time', 'lat', 'lon'], np.random.rand(365, 10, 10) + 283.15)
    ... })
    >>> monthly_ds = convert_daily_to_monthly_dataset(daily_ds)
    >>> monthly_ds.pr.attrs['units']
    'mm/mon'
    >>> monthly_ds.tasmin.attrs['units']
    'degC'
    """
    # Convert daily precipitation from kg/m2/s to mm/day
    ds['pr'] = ds.pr * 86400
    ds.pr.attrs["units"] = 'mm/day' 
    
    # Resample precipitation to monthly sum (accumulated)
    ds_precip = ds['pr'].resample(time="M").sum()
    ds_precip.attrs["units"] = 'mm/mon' 
    
    # Resample temperature to monthly mean (average)
    ds_temp = ds[['tasmin','tasmax']].resample(time="M").mean()
    
    # Convert temperature from Kelvin to Celsius
    ds_temp = ds_temp[['tasmin','tasmax']] - 273.15
    ds_temp.tasmin.attrs["units"] = 'degC'
    ds_temp.tasmax.attrs["units"] = 'degC'
        
    # Merge precipitation and temperature datasets
    ds = xr.merge([ds_precip, ds_temp])
    return ds

In [8]:
def get_dataset(esm_datastore: str, model_params: dict) -> xr.Dataset:
  """
  Return xarray.DataSet from model parameters.
  
  Parameters
  ----------
  esm_datastore : str
    URL or path to the ESM datastore catalog
  model_params : dict
    Dictionary containing model parameters with keys:
    activity_id, institution_id, table_id, experiment_id, 
    grid_label, member_id, source_id, start_year, end_year
  
  Returns
  -------
  xarray.Dataset
    Dataset containing precipitation (pr), maximum temperature (tasmax),
    and minimum temperature (tasmin) data converted to monthly time step
    and sliced to the specified time window
  
  Examples
  --------
  >>> params = {
  ...     'activity_id': 'CMIP',
  ...     'institution_id': 'NCAR',
  ...     'table_id': 'day',
  ...     'experiment_id': 'historical',
  ...     'grid_label': 'gn',
  ...     'member_id': 'r1i1p1f1',
  ...     'source_id': 'CESM2',
  ...     'start_year': '1950',
  ...     'end_year': '2014'
  ... }
  >>> ds = get_dataset("https://cadcat.s3.amazonaws.com/cae-collection.json", params)
  >>> list(ds.data_vars)
  ['pr', 'tasmax', 'tasmin']
  """
  # Open catalog of available data sets using intake-esm package
  cat = intake.open_esm_datastore(esm_datastore)
  cat_item = cat.search(
    activity_id=model_params["activity_id"],
    institution_id=model_params["institution_id"],
    table_id=model_params["table_id"], 
    variable_id=['pr','tasmax','tasmin'],
    experiment_id=model_params["experiment_id"],
    grid_label=model_params["grid_label"],
    member_id=model_params["member_id"],
    source_id=model_params["source_id"],  
  )
  
  # Add catalog item to dataset dict
  data_dict = cat_item.to_dataset_dict(
    #  xarray_open_kwargs={'consolidated': True},
    storage_options={'anon': True}
  )
  
  # Construct dataset key to retrieve from the dictionary
  key = "{}.{}.{}.{}.{}.{}".format(
      model_params['activity_id'],
      model_params['institution_id'],
      model_params['source_id'],
      model_params['experiment_id'],
      model_params['table_id'],
      model_params['grid_label'],)
  
  # Slice the dataset to the input time window.
  ds = slice_by_time_years_dataset(data_dict[key],model_params['start_year'],model_params['end_year'])
  ds = convert_daily_to_monthly_dataset(ds)
  return ds

In [9]:
def add_mask_to_dataset(mask_path: str, ds: xr.Dataset) -> xr.Dataset:
    """
    Attach the mask to input dataset ds.
    
    Parameters
    ----------
    mask_path : str
        Path to the numpy mask file (.npy format)
    ds : xarray.Dataset
        Input dataset to which the mask will be attached
    
    Returns
    -------
    xarray.Dataset
        Dataset with mask coordinate added
    
    Examples
    --------
    >>> ds = xr.Dataset({'temp': (['lat', 'lon'], np.random.rand(10, 10))})
    >>> masked_ds = add_mask_to_dataset('mask/mask.npy', ds)
    >>> 'mask' in masked_ds.coords
    True
    """
    with open(mask_path, 'rb') as f:
        mask = np.load(f, allow_pickle=True)
    ds.coords['mask'] = (('lat', 'lon'), mask)
    return ds

In [10]:
def trim_dataset_to_bbox(ds: xr.Dataset, bbox: dict) -> xr.Dataset:
    """
    Clip the dataset to a bounding box.
    
    This function sets the spatial dimensions and coordinate reference system
    for the dataset, then clips it to the specified bounding box coordinates.
    
    Parameters
    ----------
    ds : xr.Dataset
        Input xarray dataset containing spatial data with 'lon' and 'lat' dimensions
    bbox : dict
        Dictionary containing bounding box coordinates with keys:
        - 'minx': minimum longitude
        - 'miny': minimum latitude  
        - 'maxx': maximum longitude
        - 'maxy': maximum latitude
    
    Returns
    -------
    xr.Dataset
        Dataset clipped to the specified bounding box
    
    Examples
    --------
    >>> bbox = {
    ...     "maxy": 42.432494,
    ...     "miny": 34.775317,
    ...     "minx": -123.097421,
    ...     "maxx": -117.980799,
    ... }
    >>> clipped_ds = trim_dataset_to_bbox(ds, bbox)
    >>> # Returns dataset clipped to the watershed of interest
    """
    # This needs to be done for the clipping.
    ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    ds.rio.write_crs("EPSG:4326", inplace=True)

    # Get the subset of data for watershed.
    ds = ds.rio.clip_box(
        minx=bbox["minx"],
        miny=bbox["miny"],
        maxx=bbox["maxx"],
        maxy=bbox["maxy"],
    )
    return ds

In [11]:
def get_output_file_name_monthly(model_params: dict, end_part: str) -> str:
    """
    Format the output file name from model parameters.
    
    Parameters
    ----------
    model_params : dict
        Dictionary containing model parameters with keys:
        - source_id : str
            Source identifier for the model
        - experiment_id : str
            Experiment identifier (e.g., 'historical', 'ssp245')
        - member_id : str
            Member identifier (e.g., 'r1i1p1f1')
    end_part : str
        String to append at the end of the filename before the .csv extension
        
    Returns
    -------
    str
        Formatted filename in the format: {source_id}_{experiment_id}_{member_id}_{end_part}.csv
        
    Examples
    --------
    >>> params = {
    ...     'source_id': 'CESM2',
    ...     'experiment_id': 'historical',
    ...     'member_id': 'r1i1p1f1'
    ... }
    >>> get_output_file_name_monthly(params, '19FlowWeighted')
    'CESM2_historical_r1i1p1f1_19FlowWeighted.csv'
    """
    return '%s_%s_%s_%s.csv'%(model_params['source_id'],model_params['experiment_id'],model_params['member_id'],end_part)

In [12]:
def load_dataset_with_mask(esm_datastore_in: str, model_params_in: dict, mask_path: str, bbox: dict) -> xr.Dataset:
    """
    Load dataset, add mask, and trim to bounding box around area of interest.
    
    This function combines the dataset loading, mask addition, and spatial trimming
    operations into a single convenience function for processing climate data.
    
    Parameters
    ----------
    esm_datastore_in : str
        URL or path to the ESM datastore catalog
    model_params_in : dict
        Dictionary containing model parameters with keys:
        activity_id, institution_id, table_id, experiment_id, 
        grid_label, member_id, source_id, start_year, end_year
    mask_path : str
        Path to the numpy mask file (.npy format) containing regional identifiers
    bbox : dict
        Dictionary containing bounding box coordinates with keys:
        - 'minx': minimum longitude
        - 'miny': minimum latitude  
        - 'maxx': maximum longitude
        - 'maxy': maximum latitude
    
    Returns
    -------
    xr.Dataset
        Processed dataset with mask coordinate added and spatially trimmed to
        the specified bounding box, containing monthly climate data for
        precipitation (pr), maximum temperature (tasmax), and minimum 
        temperature (tasmin)
    
    Examples
    --------
    >>> params = {
    ...     'activity_id': 'CMIP',
    ...     'source_id': 'CESM2',
    ...     'experiment_id': 'historical',
    ...     'start_year': '1950',
    ...     'end_year': '2014'
    ... }
    >>> bbox = {
    ...     "minx": -123.097421,
    ...     "miny": 34.775317,
    ...     "maxx": -117.980799,
    ...     "maxy": 42.432494
    ... }
    >>> ds = load_dataset_with_mask(esm_datastore, params, 'mask/mask.npy', bbox)
    >>> 'mask' in ds.coords
    True
    """
    ds = get_dataset(esm_datastore_in, model_params_in)
    ds = add_mask_to_dataset(mask_path, ds)
    ds = trim_dataset_to_bbox(ds, bbox)
    return ds
    

In [13]:
def get_df_map_mask(id_region: int, ds: xr.Dataset, use_full_mask: bool = False) -> pd.DataFrame:
    """
    Returns a dataframe for the specified region ID from the masked dataset.
    
    This function extracts climate data for a specific region based on the mask values,
    computes spatial averages, and formats the results into a pandas DataFrame with
    monthly climate statistics.
    
    Parameters
    ----------
    id_region : int
        Region identifier to extract from the mask. Set to -1 and use_full_mask=True
        to get the entire domain excluding the specified region.
    ds : xarray.Dataset
        Input dataset containing climate variables (pr, tasmin, tasmax) and a mask
        coordinate with regional identifiers.
    use_full_mask : bool, optional
        If True, extracts data where mask != id_region (excludes the region).
        If False, extracts data where mask == id_region (includes only the region).
        Default is False.
    
    Returns
    -------
    pandas.DataFrame
        DataFrame containing monthly climate data with columns:
        - time: datetime index reset as column
        - Year: year extracted from time index
        - Month: month number (1-12)
        - Pr (mm): monthly precipitation in mm/mon
        - Tasmax (degC): monthly maximum temperature in degC
        - Tasmin (degC): monthly minimum temperature in degC
        - Tave (degC): monthly average temperature in degC
    
    Notes
    -----
    The function computes spatial averages over latitude and longitude dimensions
    for each time step, skipping NaN values. Temperature average is calculated
    as the mean of tasmax and tasmin.
    
    Examples
    --------
    >>> # Extract data for region 5
    >>> df = get_df_map_mask(5, ds)
    >>> df.columns.tolist()
    ['time', 'Year', 'Month', 'Pr (mm)', 'Tasmax (degC)', 'Tasmin (degC)', 'Tave (degC)']
    
    >>> # Extract data for entire domain excluding region 5
    >>> df_full = get_df_map_mask(5, ds, use_full_mask=True)
    """
    if use_full_mask:
        map_data = ds.where(ds.mask != id_region)
    else:
        map_data = ds.where(ds.mask == id_region)

    results_precip = map_data.pr.mean(['lat','lon'],skipna=True)
    results_precip.attrs["units"]  = 'mm/mon'

    results_tasmin = map_data.tasmin.mean(['lat','lon'],skipna=True)
    results_tasmin.attrs["units"]  = 'degC'

    results_tasmax = map_data.tasmax.mean(['lat','lon'],skipna=True)
    results_tasmax.attrs["units"]  = 'degC'

    ds_all= xr.merge([results_precip,results_tasmax,results_tasmin])
    df = ds_all.to_dataframe().reset_index()

    df.drop('spatial_ref',axis=1, inplace=True)

    df['Year'] = df['time'].dt.strftime('%Y')
    df['Month'] = df['time'].dt.month
    df['Tave (degC)'] = df[['tasmax','tasmin']].mean(axis=1)

    df.rename({'pr': 'Pr (mm)','tasmax': 'Tasmax (degC)','tasmin' : 'Tasmin (degC)'}, axis=1,inplace=True)

    df_r = df.iloc[:,[3,4,7,0,1,2,5,6]]
    df_n = df_r.reset_index()

    return df_n

In [14]:
def get_weighted_dataframe(df_in: pd.DataFrame, weighting_factor: float) -> pd.DataFrame:
    """
    Computes the weighted dataframe from df_in and returns the resulting dataframe.
    
    This function applies a weighting factor to climate variables in the input
    dataframe by multiplying precipitation and temperature values by the specified
    weighting factor.
    
    Parameters
    ----------
    df_in : pd.DataFrame
        Input dataframe containing climate data with columns:
        - 'Pr (mm)': precipitation in mm/mon
        - 'Tasmax (degC)': maximum temperature in degC
        - 'Tasmin (degC)': minimum temperature in degC
        - 'Tave (degC)': average temperature in degC
    weighting_factor : float
        Multiplicative factor to apply to climate variables. Typically represents
        area ratio or flow ratio for basin weighting calculations.
    
    Returns
    -------
    pd.DataFrame
        Dataframe with weighted climate values. All climate variables are
        multiplied by the weighting factor while preserving other columns.
    
    Notes
    -----
    The function modifies the input dataframe in-place and returns it.
    For area-weighted calculations, weighting_factor represents the area ratio
    of each sub-basin. For flow-weighted calculations, it represents the flow ratio.
    
    Examples
    --------
    >>> df = pd.DataFrame({
    ...     'Pr (mm)': [100, 150, 200],
    ...     'Tasmax (degC)': [25, 30, 35],
    ...     'Tasmin (degC)': [10, 15, 20],
    ...     'Tave (degC)': [17.5, 22.5, 27.5]
    ... })
    >>> weighted_df = get_weighted_dataframe(df, 0.3)
    >>> weighted_df['Pr (mm)'].tolist()
    [30.0, 45.0, 60.0]
    """
    df_in['Pr (mm)'] = df_in['Pr (mm)'] * weighting_factor
    df_in['Tasmax (degC)'] = df_in['Tasmax (degC)'] * weighting_factor
    df_in['Tasmin (degC)'] = df_in['Tasmin (degC)'] * weighting_factor
    df_in['Tave (degC)'] = df_in['Tave (degC)'] * weighting_factor
    return df_in

In [15]:
def get_sum_dataframes(df_in: pd.DataFrame, df_to_add: pd.DataFrame) -> pd.DataFrame:
    """
    Adds the weighted results from df_to_add to the df_in and returns the resulting dataframe.
    
    This function performs element-wise addition of climate variables between two dataframes,
    filling missing values with 0. It's used to aggregate weighted climate data across
    multiple regions or time periods.
    
    Parameters
    ----------
    df_in : pd.DataFrame
        Base dataframe containing climate data with columns:
        - 'Pr (mm)': precipitation in mm/mon
        - 'Tasmax (degC)': maximum temperature in degC
        - 'Tasmin (degC)': minimum temperature in degC
        - 'Tave (degC)': average temperature in degC
    df_to_add : pd.DataFrame
        Dataframe to add to df_in, containing the same climate columns
        
    Returns
    -------
    pd.DataFrame
        Dataframe with summed climate values. The function modifies df_in in-place
        and returns it with climate variables added from df_to_add.
        
    Notes
    -----
    The function uses pandas.Series.add() with fill_value=0 to handle missing values
    during the addition operation. This ensures that NaN values are treated as 0
    in the summation.
    
    Examples
    --------
    >>> df1 = pd.DataFrame({
    ...     'Pr (mm)': [100, 150, 200],
    ...     'Tasmax (degC)': [25, 30, 35],
    ...     'Tasmin (degC)': [10, 15, 20],
    ...     'Tave (degC)': [17.5, 22.5, 27.5]
    ... })
    >>> df2 = pd.DataFrame({
    ...     'Pr (mm)': [50, 75, 100],
    ...     'Tasmax (degC)': [5, 7, 10],
    ...     'Tasmin (degC)': [2, 3, 5],
    ...     'Tave (degC)': [3.5, 5, 7.5]
    ... })
    >>> result = get_sum_dataframes(df1, df2)
    >>> result['Pr (mm)'].tolist()
    [150.0, 225.0, 300.0]
    """
    df_in['Pr (mm)'] = df_in['Pr (mm)'].add(df_to_add['Pr (mm)'], fill_value=0)
    df_in['Tasmax (degC)'] = df_in['Tasmax (degC)'].add(df_to_add['Tasmax (degC)'], fill_value=0)
    df_in['Tasmin (degC)'] = df_in['Tasmin (degC)'].add(df_to_add['Tasmin (degC)'], fill_value=0)
    df_in['Tave (degC)'] = df_in['Tave (degC)'].add(df_to_add['Tave (degC)'], fill_value=0)
    return df_in

In [16]:
def get_monthly_rolling_ave(dict_df_weighted_all: dict, average_over_years: int, append_name: str) -> dict:
    """
    Calculate rolling average for each SSP (Shared Socioeconomic Pathway) scenario.
    
    This function processes historical and SSP climate data to compute rolling averages
    over a specified number of years. It matches historical data with corresponding
    SSP scenarios and calculates monthly averages for each rolling window.
    
    Parameters
    ----------
    dict_df_weighted_all : dict
        Dictionary with filename keys and weighted dataframe values containing
        climate data. Filenames should follow format: 
        {source_id}_{experiment_id}_{member_id}_{suffix}.csv
    average_over_years : int
        Number of years to average over for rolling calculation (typically 30)
    append_name : str
        String to append to output filenames (e.g., '30yrAve')
        
    Returns
    -------
    dict
        Dictionary with filename keys and rolling average dataframe values.
        Output filenames follow format: {source_id}_{member_id}_{ssp_experiment}_{append_name}.csv
        Each dataframe contains columns:
        - 'Year Range': string showing start-end years of rolling window
        - 'Year (30y start)': start year of rolling window
        - 'Month': month number (1-12)
        - 'Pr (mm)': monthly average precipitation
        - 'Tasmax (degC)': monthly average maximum temperature
        - 'Tasmin (degC)': monthly average minimum temperature
        - 'Tave (degC)': monthly average temperature
        
    Notes
    -----
    The function expects historical data to be paired with SSP scenarios for the
    same GCM and realization. Rolling averages are calculated from 1950 to 2072
    using the specified averaging window.
    
    Examples
    --------
    >>> weighted_data = {
    ...     'CESM2_historical_r1i1p1f1_19FlowWeighted.csv': df_hist,
    ...     'CESM2_ssp245_r1i1p1f1_19FlowWeighted.csv': df_ssp
    ... }
    >>> rolling_dict = get_monthly_rolling_ave(weighted_data, 30, '30yrAve')
    >>> list(rolling_dict.keys())
    ['CESM2_r1i1p1f1_ssp245_30yrAve.csv']
    """
    dict_fil_rolling_df = {}  # Dict with filename keys and rolling average dataframe as value.
    dict_gcm_hist_realization = {} # Dict with Historical dataframes.
    dict_gcm_other_realization = {} # Dict with SSP dataframes.
    
    #Key is the formated file name and value is dataframe.
    for key_file, value_df in dict_df_weighted_all.items():
        lst_file_parts = key_file.split('_')
        if 'historical' in key_file:
            if not lst_file_parts[0] in dict_gcm_hist_realization:
                dict_gcm_hist_realization[lst_file_parts[0]] = {}
            if not lst_file_parts[1] in dict_gcm_hist_realization[lst_file_parts[0]]:
                dict_gcm_hist_realization[lst_file_parts[0]][lst_file_parts[1]] = {}
            dict_gcm_hist_realization[lst_file_parts[0]][lst_file_parts[1]][lst_file_parts[2]] = value_df
        else:
            if not lst_file_parts[0] in dict_gcm_other_realization:
                dict_gcm_other_realization[lst_file_parts[0]] = {}
            if not lst_file_parts[1] in dict_gcm_other_realization[lst_file_parts[0]]:
                dict_gcm_other_realization[lst_file_parts[0]][lst_file_parts[1]] = {}
            dict_gcm_other_realization[lst_file_parts[0]][lst_file_parts[1]][lst_file_parts[2]]=value_df      
    
    # Do rolling average and output with dictionary key as filename and values as rolling average dataframe.
    df_rolling = pd.DataFrame
    start_year = 1950
    end_year = 2072
    for key_gcm in dict_gcm_hist_realization:
        for key_ssp in dict_gcm_hist_realization[key_gcm]:
            for key_relization in dict_gcm_hist_realization[key_gcm][key_ssp]:
                df_history = dict_gcm_hist_realization[key_gcm][key_ssp][key_relization]
                for key_ssp_other in dict_gcm_other_realization[key_gcm]:
                    if not key_relization in dict_gcm_other_realization[key_gcm][key_ssp_other]: continue
                    df_ssp = dict_gcm_other_realization[key_gcm][key_ssp_other][key_relization]
                    df_out = None
                    df_rolling = pd.concat([df_history,df_ssp], axis=0)
                    df_rolling.drop('time' , axis=1, inplace=True)  
                    for currentYear in range(start_year,end_year):
                        year_30 = currentYear + average_over_years
                        df30year = df_rolling[(df_rolling['Year'].astype(int) >= currentYear) & (df_rolling['Year'].astype(int) < year_30)]
                        dftemp = df30year.groupby(df30year.Month, as_index=False, sort=True)[['Pr (mm)','Tasmax (degC)','Tasmin (degC)','Tave (degC)']].mean().reset_index()
                        dftemp.insert(loc=0,column="Year (30y start)",value=currentYear)
                        dftemp.insert(loc=0,column="Year Range",value='%s-%s'%(currentYear,year_30-1))
                        if df_out is None:
                            df_out = dftemp.copy(deep=True)
                        else:
                            df_out = pd.concat([df_out,dftemp], axis=0)
                    df_out.drop('index', axis=1, inplace=True) 
                    file_out = '%s_%s_%s_%s.csv'%(key_gcm,key_relization,key_ssp_other,append_name)
                    dict_fil_rolling_df[file_out] = df_out
    return dict_fil_rolling_df

The loop goes through all GCMs and writes the individual subbasin, the area weighted, and the flow weighted to results dictionaries with filname as the key and value equal to result dataframe. 

In [17]:
region_dict = get_region_dict()
# main loop
all_model_params = get_model_params(run_list_path)

#Defile output dicts.
results_dict = {}
flow_weighted_results_dict = {}
area_weighted_basin_results_dict = {}
for model_params in all_model_params:
    #Add masking to the dataset.
    ds = load_dataset_with_mask(esm_datastore, model_params, mask_path,bbox)
   
    key = "{}.{}.{}.{}.{}.{}".format(
        model_params['activity_id'],
        model_params['institution_id'],
        model_params['source_id'],
        model_params['experiment_id'],
        model_params['table_id'],
        model_params['grid_label'],)
    #Force load the dataset.
    print('Loading: %s'%key)
    ds = ds.compute()
        
    df_w = None
    df_a = None
    df_nw = None
    for id_region, v in region_dict.items():     
        # Get this regions results
        df_n = get_df_map_mask(id_region,ds)
        output_filename = get_output_file_name_monthly(model_params, '%s-19'%'{:02d}'.format(id_region))

        df_out = df_n.drop('time' , axis=1)
        results_dict[output_filename] = df_out
                
        # Get Area Weighted dataframe
        weighting_factor = v['area_ratio']
        df_weighted_a = get_weighted_dataframe(df_n.copy(deep=True),weighting_factor)
        if df_a is None:
            df_a = df_weighted_a.copy(deep=True)
        else:
            df_a = get_sum_dataframes(df_a,df_weighted_a)
            
        # Get Flow Weighted results 
        weighting_factor = v['flow_ratio']
        df_weighted = get_weighted_dataframe(df_n.copy(deep=True),weighting_factor)
        if df_w is None:
            df_w = df_weighted.copy(deep=True)
        else:
            df_w = get_sum_dataframes(df_w,df_weighted)
    print('Processed %s...'%get_output_file_name_monthly(model_params,'').replace('.csv',''))       
    
    #Add weighted dataframes to output.
    output_filename = get_output_file_name_monthly(model_params, "19FlowWeighted")
    flow_weighted_results_dict[output_filename] = df_w
    output_filename = get_output_file_name_monthly(model_params, "19AreaWeighted")
    area_weighted_basin_results_dict[output_filename] = df_a


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


2025-08-05 12:50:01,470 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,476 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,482 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,483 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,485 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,485 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,486 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,488 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,488 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 12:50:01,491 - distributed.core - INFO - Con

Loading: LOCA2.UCSD.MIROC6.historical.day.d03


2025-08-05 12:51:50,426 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:33695
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/comm/tcp.py", line 226, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/worker.py", line 2073, in gather_dep
    response = await get_data_from_worker(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/worker.py", line 2879, in get_data_from_worker
    response = await send_recv(
               ^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distri

Processed MIROC6_historical_r4i1p1f1_...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


...

Processed MRI-ESM2-0_ssp370_r4i1p1f1_...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/groupers.py:498: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/groupers.py:498: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


Loading: LOCA2.UCSD.MRI-ESM2-0.ssp370.day.d03


2025-08-05 13:52:21,431 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f383092c320>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7f381fcba990>, 982112.487675132)])']
connector: <aiohttp.connector.TCPConnector object at 0x7f3834355a60>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f6c7f5359a0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7f6c6fa77590>, 982108.217174798)])']
connector: <aiohttp.connector.TCPConnector object at 0x7f6c7f6041a0>
2025-08-05 13:52:22,415 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2025-08-05 13:52:22,441 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has been closed.
2025-08-05 13:52:22,442 - distributed.core - INFO - Connection to tcp://127.0.0.1:45569 has 

Processed MRI-ESM2-0_ssp370_r5i1p1f1_...


In [18]:
#Write to ouput.
zip_path = os.path.join(output_folder, file_zip)

dict_rolling_flow_weighted = get_monthly_rolling_ave(flow_weighted_results_dict,30,'30yrAve')
dict_rolling_area_weighted = get_monthly_rolling_ave(area_weighted_basin_results_dict,30,'30yrAve')

with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
    for k, v in results_dict.items():
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        fileout = dir_individual + '/' + k
        zf.writestr(fileout, text_stream.getvalue())

    for k, v in flow_weighted_results_dict.items():
        fileout = dir_flow_weighted + '/' + k
        fileout_raw = dir_flow_weighted + '/Raw/' + k
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout_raw, text_stream.getvalue())
        v.drop('time' , axis=1, inplace=True)
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout, text_stream.getvalue())

    for k, v in area_weighted_basin_results_dict.items():
        fileout = dir_area_weighted + '/' + k
        fileout_raw = dir_area_weighted + '/Raw/' + k
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout_raw, text_stream.getvalue())
        v.drop('time' , axis=1, inplace=True)
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout, text_stream.getvalue())

    for k, v in dict_rolling_area_weighted.items():
        fileout = dir_area_weighted_rolling + '/' + k
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout, text_stream.getvalue())
        
    for k, v in dict_rolling_flow_weighted.items():
        fileout = dir_flow_weighted_rolling + '/' + k
        text_stream = StringIO();
        v.to_csv(text_stream, index=False)
        zf.writestr(fileout, text_stream.getvalue())
    

In [19]:
client.close()